# Expectation Maximization - GMMs NumPy

En éste notebook vamos a aplicar los conceptos teóricos del método iterativo de expectation maximization para estimar los parámetros de un Gaussian Mixture Model en NumPy.

## Importamos las Liberías

In [1]:
import numpy as np
import seaborn as sns
from scipy.stats import multivariate_normal
from scipy.stats import norm

In [2]:
import sys  
sys.path.insert(0, '../../clase_3/ejercicios/src')
sys.path.insert(0, '../../clase_1/ejercicios/src')

In [3]:
from models import BaseModel
from metrics import Accuracy, Precision, Recall
from k_means_numpy import k_means, k_means_classify

## Expectation Maximization para GMM

A continuación, vamos a introducir los building blocks de cada uno de los pasos de EM para los Gaussian Mixture Models. Finalizaremos la sección armando un modelo de GMM con la misma estructura de BaseModel.

### 1. Inicialización de Parámetros

In [ ]:
# El primer paso de EM es inicializar los parámetros del modelo
# (probabilidades, medias y covarianzas de los componentes)

# Indicamos la cantidad de componentes
k = 2

# Inicializamos y normalizamos las probabilidades (kx1)
p = np.random.uniform(0, 1, (k, 1))
p = p / np.sum(p, axis=0)

# Inicializamos las medias dentro del rango de los datos (kx1)
means = np.random.uniform(np.min(X), np.max(X), (k, 1))

# Inicalizamos la varianza a partir de los datos y las medias escogidas (kx1).
# Estamos usando broadcasting para el cálculo.
covariance = np.sum((np.hstack((X, X))- means.T)**2, axis=0)/(X.shape[0]-1)
covariance = covariance.reshape(-1, 1)

### 2. E-Step: cálculo de responsabilidades (a posteriori)

Determinen las responsabilidades $\omega_{k}$ para cada componente y cada muestra. Para ello, deben calcular la pdf de la distribución normal multivariada. Hint: usen multivariate_normal.pdf de Scipy. $ \omega_{k} = \frac{\pi_{k} * \mathcal{N}(x_n | \mu_k, \Sigma_k  ) }{\sum_{j}^{}\pi_{j} * \mathcal{N}(x_n | \mu_j, \Sigma_j  )} $

In [ ]:
# Armen una matriz para almacenar los resultados de la pdf de cada muestra para cada componente (nxk)

Nij = np.zeros((n, k))

for j in range(k):
    Nij[:, j] = multivariate_normal.pdf(X, means[j], covariance[j])

# Armen una matriz para almacenar las responsabilidades de cada muestra y cada componente (nxk)

Eij = np.zeros((n, k))

for j in range(k):
    Eij[:, j] = (p[j] * Nij[:, j]) / (Nij @ p)[:, 0]

### 3. M-Step: redeterminación de parámetros

$\mu_{j} = \frac{\sum_{i=1}^{N}\omega_{j}^{(i)}x^{(i)}}{\sum_{i=1}^{N}\omega_{j}^{(i)}}$

$\pi_{j} = \frac{1}{N}\sum_{i=1}^{N}\omega_{j}^{(i)}$ 

$\Sigma_{j} = \frac{\sum_{i=1}^{N}\omega_{j}^{(i)}(x^{(i)}-\mu_j)(x^{(i)}-\mu_j)^T}{\sum_{i=1}^{N}\omega_{j}^{(i)}}$

In [ ]:
# Con los valores calculados de las responsabilidades, recalcular los parámetros

# Redeterminar las medias
means[j] = (Eij[:, j].dot(X)) / np.sum(Eij[:, j], axis=0)

# Redeterminar la covarianza
covariance[j] = Eij[:, j].dot((X - means[j]) * (X - means[j])) / np.sum(Eij[:, j])

# Redeterminar las probabilidades
p[j] = np.mean(Eij[:, j])

### Método fit

Con los bloques anteriores, armar un método fit que reciba X, y y la cantidad de clusters y retorne como resultado los parámetros del modelo de GMM (probabilidades, medias y covarianzas de cada componente). Al ser un algoritmo iterativo, es importante determinar un criterio de parada. Se suele utilizar un número máximos de iteraciones MAX_ITER y simultáneamente se  evalúa la diferencia entre los parámetros de la iteración actual y la anterior, para ver si se modifican por sobre determinada tolerancia. 

In [ ]:
def fit(X, y, k):
    # Paso 1: Inicialización parámetros
    # Loop con criterio de parada (MAX_ITER, tol):
        # Paso 2: E-Step
        # Paso 3: M-Step
    return NotImplemented

### Método predict

Una vez determinados los parámetros de cada componente del GMM, cuando recibimos una nueva muestra, debemos usar los mísmos para calcular las responsabilidades (probabilidades a posteriori) de cada componente para esa muestra y devolver como "cluster", el componente cuya probabilidad sea mayor.

In [ ]:
def predict(X):
    
    # Calculamos las responsabilidades de cada muestra para cada componente
    N = np.zeros((X.shape[0], k))
    E = np.zeros((X.shape[0], k))
    
    for i in range(k):
        N[:, i] = multivariate_normal.pdf(X, self.model['mu'][i, 0], self.model['cov'][i, 0])
    
    for i in range(k):
        E[:, i] = (self.model['p'][i, 0] * N[:, i]) / (N @ self.model['p'])[:, 0]
    
    # Devolvemos como cluster, el componente de máxima probabilidad para cada muestra
    idx = np.argmax(E, axis=1)

## Modelo Final

In [ ]:
class EMScalar(BaseModel):
    
    def fit(self, X):

        MAX_IT = 500

        # Dimensions
        n = X.shape[0]
        k = 2

        # Parameters initialization
        p = np.random.uniform(0, 1, (k, 1))
        p = p / np.sum(p, axis=0)
        
        means = np.random.uniform(np.min(X), np.max(X), (k, 1))
        
        covariance = np.sum((np.hstack((X, X))-means.T)**2, axis=0)/(X.shape[0]-1)
        covariance = covariance.reshape(-1, 1)
        
        Nij = np.zeros((n, k))
        Eij = np.zeros((n, k))
        Eij_ant = np.zeros((n, k))

        i = 0
        delta = False
        tol = 1E-5
        for j in range(k):
            Nij[:, j] = multivariate_normal.pdf(X, means[j], covariance[j])

        # Execution Loop
        while not (delta or i > MAX_IT):
            Eij_ant[:, :] = Eij
            for j in range(k):
                Eij[:, j] = (p[j] * Nij[:, j]) / (Nij @ p)[:, 0]
                means[j] = (Eij[:, j].dot(X)) / np.sum(Eij[:, j], axis=0)
                covariance[j] = Eij[:, j].dot((X - means[j]) * (X - means[j])) / np.sum(Eij[:, j])
                p[j] = np.mean(Eij[:, j])
                Nij[:, j] = multivariate_normal.pdf(X, means[j], covariance[j])
            delta = np.allclose(Eij_ant, Eij, rtol=tol)
            i = i + 1
        idx = np.argsort(means[:, 0], axis=0)
        self.model = {'mu': means[idx, :], 'cov': covariance[idx, :], 'p': p[idx, :]}

    def predict(self, X):
        k = self.model['mu'].shape[0]
        N = np.zeros((X.shape[0], k))
        E = np.zeros((X.shape[0], k))

        for i in range(k):
            N[:, i] = multivariate_normal.pdf(X, self.model['mu'][i, 0], self.model['cov'][i, 0])
        for i in range(k):
            E[:, i] = (self.model['p'][i, 0] * N[:, i]) / (N @ self.model['p'])[:, 0]
        idx = np.argmax(E, axis=1)
        return idx